# Azure ML - Compute Instances

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [4]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(
    credential=DefaultAzureCredential(), 
    suscription_id=AZURE_SUSCRIPION_ID, 
    resource_group_name=AZURE_ML_RESOURCE_GRPU_ID, 
    workspace_name=workspace_name
)

## Compute Instances

Va,mos a volver a ver la arquitectura de `Azure ML`

![Azure ML architecture concepts](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/azureml-architecture-concepts.webp)

Ya hemos visto cómo crear un `Azure Data Storage` y vincularlo a `Azure ML`, cómo crear un `Datastore` y cómo crear un `Dataset`. Ahora necesitamos una máquina dónde empezar a entrenar

Para ello tenemos tres opciones

 * `Compute Instances`: Son máquinas virtuales que se pueden usar para desarrollo y pruebas. No tenemos que preocuparnos del sistema operativo, ni de la instalación de librerías, ni de nada. Son máquinas virtuales que se pueden apagar y encender
 * `Compute Clusters`: Son como los `Compute Instances` pero en lugar de ser una máquina virtual, es un cluster de máquinas virtuales. Se pueden usar para entrenar modelos en paralelo
 * `Compute Targets`: En este caso, esto son máquinas externas, que pueden ser otras máquinas de `Azure`, de otro cloud, o incluso una máquina local.

En este caso vamos a ver cómo crear un `Compute Instance` tanto desde la interfaz web, como desde la CLI de `Azure ML` y como hacerlo con el SDK de Python

#### Crear un `Compute instance` desde la interfaz gráfica

En `Azure ML studio` deberíamos tener el `Workspace` `Azure ML Workspace GUI`, así que entramos en él y pinchamos en el botón `New` y seleccionamos `Compute instance`
 * Le ponemos un nombre, en mi caso le pondré `compute-instance-GUI`
 * Seleccionamos el tipo de máquina, si `CPU` o `GPU`, en mi caso selecciono `GPU` para poder entrenar modelos
 * Seleccionamos el tipo de `GPU` que queremos, espero que cuando leas esto los filtros sean mejores y puedas filtrar por `VRAM`, pero cuando lo estoy escribiendo no está ese filtro básico. En mi caso, como no voy a hacer entrenamientos muy grandes elijo la `Standard_NC4as_T4_v3` que tiene una `Tesla T4` con `16GB` de `VRAM`
 * Pulsamos `Next` y en la siguiente pantalla dejamos la configuración de autoapagado después de 60 minutos de inactividad, por si acaso
 * El resto de opciones las dejamos como están y creamos la instancia, tardará unos minutos

Desafortunadamente por defecto no vamos a tener quota de máquinas con GPU. Por lo que aunque hayamos creado una máquina con GPU, no vamos a poder usarla. Más adelante hablaremos de esto

Cuando se crea hay una zona llamada `Applications` donde podemos elegir si ejecutar la instancia en `JupyterLab`, `Jupyter`, `vscode` en la web o `vscode` en local.

Por otro lado, cuando se crea la instancia, se crea encendida, así que entramos y la apagamos para que no genere costes. Cuando entras en la instancia hay un botón q`Stop` para apagarla

#### Crear un `Compute instance` con el CLI de Azure ML

Creamos la `Compute instance` con el siguiente comando

In [ ]:
!GROUP='rg-azure-ml' && \
LOCATION='francecentral' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
SIZE="STANDARD_E4DS_V4" && \
TYPE="computeinstance" && \
az ml compute create --name $NAME --size $SIZE --type $TYPE --workspace-name $WORKSPACE --resource-group $GROUP

{- Finished ..
  "created_on": "2024-10-15T08:51:45.785449+0000",
  "enable_node_public_ip": true,
  "enable_os_patching": false,
  "enable_root_access": true,
  "enable_sso": true,
  "id": "/subscriptions/c1ffdd1c-ecfc-4119-bf19-e207f9a4bcaf/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-CLI/computes/compute-instance-CLI",
  "last_operation": {
    "operation_name": "Create",
    "operation_status": "Succeeded",
    "operation_time": "2024-10-15T08:51:54.183Z",
    "operation_trigger": "User"
  },
  "location": "francecentral",
  "name": "compute-instance-CLI",
  "network_settings": {
    "private_ip_address": "10.0.0.4",
    "public_ip_address": "4.233.64.249"
  },
  "os_image_metadata": {
    "current_image_version": "24.09.23",
    "is_latest_os_image_version": true,
    "latest_image_version": "24.09.23"
  },
  "provisioning_state": "Succeeded",
  "release_quota_on_stop": false,
  "resourceGroup": "rg-azure-ml",
  "services": [

Desafortunadamente por defecto no vamos a tener quota de máquinas con GPU. Por lo que aunque hayamos creado una máquina con GPU, no vamos a poder usarla. Más adelante hablaremos de esto

#### Crear un `Compute instance` con el SDK de Python

Creamos la `Compute instance` con el siguiente código. Lo llamo `compute-instance-Python` y no `compute-instance-Python-SDK` porque el nombre tienen que tener 24 caracteres como máximo. Además, este `Compute instance` lo voy a crear con una GPU `Standard_NC8as_T4_v3` porque no me deja crear más `Compute instance`s con la GPU que he elegido antes en la misma suscripción

In [5]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"
size = "STANDARD_E4DS_V4"
time_before_shutdown_minutes = "30"

compute_instance = ComputeInstance(name=compute_instance_name, size=size, idle_time_before_shutdown_minutes=time_before_shutdown_minutes)

result = ml_client.begin_create_or_update(compute_instance).result()

In [6]:
result

ComputeInstance({'state': 'Running', 'last_operation': {'operation_name': 'Create', 'operation_time': '2024-10-28T16:48:19.312Z', 'operation_status': 'Succeeded', 'operation_trigger': 'User'}, 'os_image_metadata': <azure.ai.ml.entities._compute._image_metadata.ImageMetadata object at 0x7cc79e3cee90>, 'services': [{'display_name': 'Jupyter', 'endpoint_uri': 'https://compute-instance-python.francecentral.instances.azureml.ms/tree/'}, {'display_name': 'Jupyter Lab', 'endpoint_uri': 'https://compute-instance-python.francecentral.instances.azureml.ms/lab'}], 'type': 'computeinstance', 'created_on': '2024-10-28T16:48:10.527349+0000', 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'compute-instance-Python', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-Python-SDK/computes/compute-

Desafortunadamente por defecto no vamos a tener quota de máquinas con GPU. Por lo que aunque hayamos creado una máquina con GPU, no vamos a poder usarla. Más adelante hablaremos de esto

### Administrar un `Compute instance` de Azure ML

#### Administrar un `Compute instance` desde la interfaz gráfica

Entramos en la sección `Compute` del `Workspace` y pinchamos en la `Compute instance` que queramos administrar y a través de la interfaz gráfica podemos hacer lo que queramos

#### Administrar un `Compute instance` con el CLI de Azure ML

Podemos realizar varias operaciones básicas en la `COmpute instance` con el CLI de Azure ML

##### Parar un `Compute instance` con el CLI de Azure ML

In [ ]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
az ml compute stop --name $NAME --workspace-name $WORKSPACE --resource-group $GROUP

inished ..

##### Iniciar un `Compute instance` con el CLI de Azure ML

In [ ]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
az ml compute start --name $NAME --workspace-name $WORKSPACE --resource-group $GROUP

 / Finished ..

##### Reiniciar un `Compute instance` con el CLI de Azure ML

In [ ]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
az ml compute restart --name $NAME --workspace-name $WORKSPACE --resource-group $GROUP

 / Finished ..

##### Eliminar un `Compute instance` con el CLI de Azure ML

In [ ]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
az ml compute delete --name $NAME --workspace-name $WORKSPACE --resource-group $GROUP

Are you sure you want to perform this operation? (y/n): ^C


Respondo que no porque voy a seguir trabajando con el `Compute instance`

##### Observar un `Compute instance` con el CLI de Azure ML

In [ ]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
az ml compute show --name $NAME --workspace-name $WORKSPACE --resource-group $GROUP

{
  "created_on": "2024-10-14T10:47:41.472829+0000",
  "enable_node_public_ip": true,
  "enable_os_patching": false,
  "enable_root_access": true,
  "enable_sso": true,
  "id": "/subscriptions/c1ffdd1c-ecfc-4119-bf19-e207f9a4bcaf/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-CLI/computes/compute-instance-CLI",
  "last_operation": {
    "operation_name": "Start",
    "operation_status": "Succeeded",
    "operation_time": "2024-10-14T11:12:50.323Z",
    "operation_trigger": "User"
  },
  "location": "francecentral",
  "name": "compute-instance-CLI",
  "network_settings": {
    "private_ip_address": "10.0.0.4",
    "public_ip_address": "52.143.163.77"
  },
  "os_image_metadata": {
    "current_image_version": "24.09.23",
    "is_latest_os_image_version": true,
    "latest_image_version": "24.09.23"
  },
  "provisioning_state": "Succeeded",
  "release_quota_on_stop": false,
  "resourceGroup": "rg-azure-ml",
  "services": [
    {
      

 > ¡Asegurate de pararlo si no vas a aseguir trabajando ahora!

#### Administrar un `Compute instance` con el SDK de Python

##### Estado de un `Compute instance` con el SDK de Python

In [7]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"

compute_instance_state = ml_client.compute.get(compute_instance_name)
print(compute_instance_state)

created_on: 2024-10-28T16:48:10.527349+0000
enable_node_public_ip: true
enable_os_patching: false
enable_root_access: true
enable_sso: true
id: /subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-Python-SDK/computes/compute-instance-Python
idle_time_before_shutdown: PT30M
idle_time_before_shutdown_minutes: 30
last_operation:
  operation_name: Create
  operation_status: Succeeded
  operation_time: '2024-10-28T16:48:19.312Z'
  operation_trigger: User
location: francecentral
name: compute-instance-Python
network_settings:
  private_ip_address: 10.0.0.4
  public_ip_address: 4.178.173.186
os_image_metadata:
  current_image_version: 24.09.23
  is_latest_os_image_version: true
  latest_image_version: 24.09.23
provisioning_state: Succeeded
release_quota_on_stop: false
services:
- display_name: Jupyter
  endpoint_uri: https://compute-instance-python.francecentral.instances.azureml.ms/tree/
- dis

##### Parar un `Compute instance` con el SDK de Python

In [10]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"

ml_client.compute.begin_stop(compute_instance_name).wait()

##### Iniciar un `Compute instance` con el SDK de Python

In [9]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"

ml_client.compute.begin_start(compute_instance_name).wait()

##### Reiniciar un `Compute instance` con el SDK de Python

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"

ml_client.compute.begin_restart(compute_instance_name).wait()

##### Eliminar un `Compute instance` con el SDK de Python

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"

ml_client.compute.begin_delete(compute_instance_name).wait()

##### Observar un `Compute instance` con el SDK de Python

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

compute_instance_name = "compute-instance-Python"

instance = ml_client.compute.get(compute_instance_name)
print(instance.os_image_metadata)

Vemos que ha devuelto un objeto de la clase `ComputeInstance`, para ver todos los parámetros de esta clase podemos entrar en [azure.ai.ml.entities.computeinstance](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.computeinstance?view=azure-python). Vamos a ver por ejemplo el nombre

In [ ]:
instance.name

'compute-instance-Python'